# Importing the dependencies

In [1]:
import numpy as np 
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder 

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

In [3]:
#printing the stop words to be removed
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# analysing the dataset

In [4]:
#loading the datasets
news_dataset= pd.read_csv("/Users/shashikanthbokka/Desktop/2-YEAR/minor project/fake_or_real_news.csv")

/var/folders/bh/n86hlp2d1cs14rcgqpt1x10h0000gn/T/ipykernel_9134/3178657495.py:2: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140) have mixed types. Specify dtype option on import or set low_memory=False.
  news_dataset= pd.read_csv("/Users/shashikanthbokka/Desktop/2-YEAR/minor project/fake_or_real_news.csv")


In [5]:
news_dataset.shape

(7795, 141)

In [6]:
#printing the first 5 rows
news_dataset.head()

,Unnamed: 0,title,text,label,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#counting null values
news_dataset.isnull().sum()

Unnamed: 0       219
title            610
text             866
label           1040
Unnamed: 4      7477
                ... 
Unnamed: 136    7794
Unnamed: 137    7794
Unnamed: 138    7794
Unnamed: 139    7794
Unnamed: 140    7794
Length: 141, dtype: int64

In [8]:
#replacing the null values with empty string
news_dataset=news_dataset.fillna('')

In [9]:
news_dataset.isnull().sum()

Unnamed: 0      0
title           0
text            0
label           0
Unnamed: 4      0
               ..
Unnamed: 136    0
Unnamed: 137    0
Unnamed: 138    0
Unnamed: 139    0
Unnamed: 140    0
Length: 141, dtype: int64

In [10]:
#merging the titel and text
news_dataset['content']=news_dataset['title']+''+news_dataset['text']+''+news_dataset['label']

/var/folders/bh/n86hlp2d1cs14rcgqpt1x10h0000gn/T/ipykernel_9134/3325400575.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  news_dataset['content']=news_dataset['title']+''+news_dataset['text']+''+news_dataset['label']


In [11]:
display(news_dataset['content'])

0       You Can Smell Hillary’s FearDaniel Greenfield,...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathyU.S...
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
7790    State Department says it can't find emails fro...
7791    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
7792    Anti-Trump Protesters Are Tools of the Oligarc...
7793    In Ethiopia, Obama seeks progress on peace, se...
7794    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 7795, dtype: object

In [12]:
#deleting the unwanted columns
del news_dataset['Unnamed: 4']
del news_dataset['Unnamed: 5']
del news_dataset['title']


In [13]:
# separting the data and label
X=news_dataset.drop(columns=('label'),axis=1)
Y=news_dataset['label']

In [14]:
print(X)
print(Y)

     Unnamed: 0                                               text Unnamed: 6  \
0          8476  Daniel Greenfield, a Shillman Journalism Fello...              
1         10294  Google Pinterest Digg Linkedin Reddit Stumbleu...              
2          3608  U.S. Secretary of State John F. Kerry said Mon...              
3         10142  — Kaydee King (@KaydeeKing) November 9, 2016 T...              
4           875  It's primary day in New York and front-runners...              
...         ...                                                ...        ...   
7790       4490  The State Department told the Republican Natio...              
7791       8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...              
7792       8622   Anti-Trump Protesters Are Tools of the Oligar...              
7793       4021  ADDIS ABABA, Ethiopia —President Obama convene...              
7794       4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...              

     Unnamed: 7 Unnamed: 8 

In [15]:
news_dataset.head()

,Unnamed: 0,text,label,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,content
0,8476,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,,,,,,,,...,,,,,,,,,,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,10294,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,,,,,,,,...,,,,,,,,,,Watch The Exact Moment Paul Ryan Committed Pol...
2,3608,U.S. Secretary of State John F. Kerry said Mon...,REAL,,,,,,,,...,,,,,,,,,,Kerry to go to Paris in gesture of sympathyU.S...
3,10142,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,,,,,,,,...,,,,,,,,,,Bernie supporters on Twitter erupt in anger ag...
4,875,It's primary day in New York and front-runners...,REAL,,,,,,,,...,,,,,,,,,,The Battle of New York: Why This Primary Matte...


# Stemming

In [16]:
import string
import re
import nltk
import pandas as pd
import numpy as np
import nltk.util as pr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
stemmer =nltk.SnowballStemmer('english')
nltk.download('stopwords')
stopword=set(stopwords.words('english'))
import warnings
warnings.filterwarnings('ignore')


[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [17]:
# port_stem=PorterStemmer()

In [18]:
# # cleaning the data
# def stemming(content):
#     stemmed_content= re.sub('[^a-zA-z]',' ', content)
#     stemmed_content=stemmed_content.lower()
#     stemmed_content=stemmed_content.split()
#     stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
#     stemmed_content=' '.join(stemmed_content)
#     return stemmed_content
    
             

In [19]:
# news_dataset['content']=news_dataset['content'].apply(stemming)

In [20]:
# print(news_dataset['content'])

In [21]:
#cleaning the data
def clean(title):
    title=str(title).lower()
    title=re.sub('\[.*?\}]','',title)
    title=re.sub('https?://\S+|www\.\S+','',title)
    title=re.sub('<.*?>+','',title)
    title=re.sub('[%s]'%re.escape(string.punctuation),'',title)
    title=re.sub('\n','',title)
    title=re.sub('\w*\d\w*','',title)
    title=[word for word in title.split(' ') if word not in stopword]
    title=" ".join(title)
    title=[stemmer.stem(word) for word in title.split(' ')]
    title=" ".join(title)
    return title
news_dataset['label']=news_dataset['label'].apply(clean)
# print(data['label'])


In [22]:
print(news_dataset['content']) 

0       You Can Smell Hillary’s FearDaniel Greenfield,...
1       Watch The Exact Moment Paul Ryan Committed Pol...
2       Kerry to go to Paris in gesture of sympathyU.S...
3       Bernie supporters on Twitter erupt in anger ag...
4       The Battle of New York: Why This Primary Matte...
                              ...                        
7790    State Department says it can't find emails fro...
7791    The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
7792    Anti-Trump Protesters Are Tools of the Oligarc...
7793    In Ethiopia, Obama seeks progress on peace, se...
7794    Jeb Bush Is Suddenly Attacking Trump. Here's W...
Name: content, Length: 7795, dtype: object


In [23]:
le = LabelEncoder()
news_dataset['label']=le.fit_transform(news_dataset.label)
news_dataset.head()

,Unnamed: 0,text,label,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,...,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,content
0,8476,"Daniel Greenfield, a Shillman Journalism Fello...",418,,,,,,,,...,,,,,,,,,,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,10294,Google Pinterest Digg Linkedin Reddit Stumbleu...,418,,,,,,,,...,,,,,,,,,,Watch The Exact Moment Paul Ryan Committed Pol...
2,3608,U.S. Secretary of State John F. Kerry said Mon...,419,,,,,,,,...,,,,,,,,,,Kerry to go to Paris in gesture of sympathyU.S...
3,10142,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",418,,,,,,,,...,,,,,,,,,,Bernie supporters on Twitter erupt in anger ag...
4,875,It's primary day in New York and front-runners...,419,,,,,,,,...,,,,,,,,,,The Battle of New York: Why This Primary Matte...


In [24]:
news_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7795 entries, 0 to 7794
Columns: 139 entries, Unnamed: 0 to content
dtypes: int64(1), object(138)
memory usage: 8.3+ MB


In [25]:
#separating the data and lae\bel
X= news_dataset['content'].values
Y= news_dataset['label'].values

In [26]:
print(X)

['You Can Smell Hillary’s FearDaniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing on radical Islam. \nIn the final stretch of the election, Hillary Rodham Clinton has gone to war with the FBI. \nThe word “unprecedented” has been thrown around so often this election that it ought to be retired. But it’s still unprecedented for the nominee of a major political party to go war with the FBI. \nBut that’s exactly what Hillary and her people have done. Coma patients just waking up now and watching an hour of CNN from their hospital beds would assume that FBI Director James Comey is Hillary’s opponent in this election. \nThe FBI is under attack by everyone from Obama to CNN. Hillary’s people have circulated a letter attacking Comey. There are currently more media hit pieces lambasting him than targeting Trump. It wouldn’t be too surprising if the Clintons or their allies were to start running attack ads against the FBI. \nThe FBI’s leadership i

In [27]:
print(Y)

[418 418 419 ... 418 419 419]


In [28]:
#converting the text to numerical data a comp can understadn
vectorizer=TfidfVectorizer()
# vectorizer.fit(X)
X=vectorizer.fit_transform(X)

    
# X=vectorizer.transform(X)

Splitting the dataset

In [29]:
X_train,X_test,Y_train, Y_test= train_test_split(X,Y, test_size=0.2,random_state=2)

In [30]:
# logistic regression
# model=LogisticRegression()
# model=DecisionTreeClassifier()
# model=LogisticRegression(random_state=0)
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=10,criterion='entropy')

In [31]:
model.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [32]:
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction, Y_train)

In [33]:
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction, Y_test)

In [34]:
a=(training_data_accuracy*100)
b=(test_data_accuracy*100)
a=str(round(a,3))
b=str(round(b,3))
print('Accuracy of training data :',a,'%')
print('Accuracy of testing data :',b, '%')

Accuracy of training data : 99.711 %
Accuracy of testing data : 80.885 %


In [35]:
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[418]
The news is Fake


In [36]:
print(Y_test[5])

418
